In [ ]:
###########
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio

# Define the range for A, B, and C
A = np.linspace(0, 2, 50)
B = np.linspace(0, 2, 50)
C = np.linspace(0, 2, 50)

# Create 3D meshgrid for A, B, and C
A_3d, B_3d, C_3d = np.meshgrid(A, B, C, indexing='ij')

# Compute inequalities
inequality_AC = A_3d + C_3d >= np.log2(np.ceil(2**A_3d))  
inequality_AB = A_3d + B_3d >= np.log2(np.ceil(2**A_3d))  
inequality_BC = 2 * A_3d + B_3d + C_3d >= 2 * np.log2(np.ceil(2**A_3d))  

# Compute the union of inequalities (valid region)
valid_region = np.logical_or.reduce([inequality_AC, inequality_AB])

# Extract valid region points
A_valid, B_valid, C_valid = A_3d[valid_region], B_3d[valid_region], C_3d[valid_region]

# Extract edges
edges_AC = (np.roll(inequality_AC[:, 0, :], 1, axis=0) ^ inequality_AC[:, 0, :]) | \
           (np.roll(inequality_AC[:, 0, :], 1, axis=1) ^ inequality_AC[:, 0, :])

edges_AB = (np.roll(inequality_AB[:, :, 0], 1, axis=0) ^ inequality_AB[:, :, 0]) | \
           (np.roll(inequality_AB[:, :, 0], 1, axis=1) ^ inequality_AB[:, :, 0])

# Extract edge points
A_edges_C, C_edges = A_3d[:, 0, :][edges_AC], C_3d[:, 0, :][edges_AC]  
A_edges_B, B_edges = A_3d[:, :, 0][edges_AB], B_3d[:, :, 0][edges_AB]  

# Extract intersection points
edges_ABC = (
    (np.roll(valid_region, 1, axis=0) ^ valid_region) |
    (np.roll(valid_region, 1, axis=1) ^ valid_region) |
    (np.roll(valid_region, 1, axis=2) ^ valid_region)
)
edges_BA = inequality_AB & edges_ABC  
edges_Ca = inequality_BC & edges_ABC  
edges_intersection = edges_BA & edges_Ca  

A_edges_intersect, B_edges_intersect, C_edges_intersect = A_3d[edges_intersection], B_3d[edges_intersection], C_3d[edges_intersection]

# Define marked point
marked_A, marked_B, marked_C = 0.188649, 0.089036, 0.372337

# Create a 3D scatter plot
fig = go.Figure()

# Add gray valid region
fig.add_trace(go.Scatter3d(
    x=C_valid, y=B_valid, z=A_valid,
    mode='markers',
    marker=dict(color='gray', size=3, opacity=0.3),
    name="Valid Region"
))

# Add black boundary for A + C ≥ log2(ceil(2^A))
# fig.add_trace(go.Scatter3d(
#     x=C_edges, y=np.zeros_like(C_edges), z=A_edges_C,
#     mode='markers',
#     marker=dict(color='black', size=5),
#     name="Boundary (A + C ≥ log2(ceil(2^A)))"
# ))

# # Add black boundary for A + B ≥ log2(ceil(2^A))
# fig.add_trace(go.Scatter3d(
#     x=np.zeros_like(B_edges), y=B_edges, z=A_edges_B,
#     mode='markers',
#     marker=dict(color='black', size=5),
#     name="Boundary (A + B ≥ log2(ceil(2^A)))"
# ))

# Add blue cross for the marked point
fig.add_trace(go.Scatter3d(
    x=[marked_C], y=[marked_B], z=[marked_A],
    mode='markers',
    marker=dict(color='blue', size=8, symbol='x'),
    name="Marked Point (0.188649, 0.089036, 0.372337)"
))

# Define additional lines
lines = [
    [(0, 0, 0), (0, 1, 0)],    # Line 1
    [(0, 0, 0), (1, 0, 0)],    # Line 2
    [(0, 1, 0), (1, 1, 0)],    # Line 3
    [(1, 0, 0), (1, 1, 0)],    # Line 4
    [(1, 1, 0), (0, 0, 1)],    # Line 5
    [(0, 0, 1), (0, 0.57, 1.02)],  # Line 6
    [(0, 0, 1), (0.57, 0, 1.02)],  # Line 7
    [(0.57, 0, 1.02), (0.57, 0.57, 1.02)],  # Line 8
    [(0, 0.57, 1.02), (0.57, 0.57, 1.02)],  # Line 9
    [(0.57, 0.57, 1.02), (0, 0, 1.59)],   # Line 10
    [(0, 1, 0), (0, 0, 1)],   # Line 11
    [(1, 0, 0), (0, 0, 1)],   # Line 12
    [(0, 0.57, 1.02), (0.57, 0.57, 1.02)],   # Line 13
    [(0, 0.57, 1.02), (0., 0., 1.59)],   # Line 14 
    [(0.57, 0, 1.02), (0., 0., 1.59)],   # Line 15  
    
    [(0, 0, 1.59), (0.42, 0, 1.59)],   # Line 16
    [(0, 0, 1.59), (0, 0.42, 1.59)],   # Line 17
    [(0.42, 0, 1.59), (0.42, 0.42, 1.59)],   # Line 18
    [(0, 0.42, 1.59), (0.42, 0.42, 1.59)],  # Line 19
    [(0.42, 0.42, 1.59), (0, 0, 2)],   # Line 20
    [(0, 0.42, 1.59), (0, 0, 2)],   # Line 21
    [(0.42, 0.42, 1.59), (0, 0, 2)],   # Line 22
    [(0.42, 0, 1.59), (0, 0, 2)]   # Line 22
]

# Add each line to the figure
for (x1, y1, z1), (x2, y2, z2) in lines:
    fig.add_trace(go.Scatter3d(
        x=[x1, x2], y=[y1, y2], z=[z1, z2],
        mode='lines',
        line=dict(color='black', width=3),  # Adjust width for visibility
        showlegend=False 
    ))

# Set axis labels and layout
fig.update_layout(
    scene=dict(
        xaxis_title="C",
        yaxis_title="B",
        zaxis_title="A"
    ),
    title="3D Visualization with Additional Lines",
    width=800, height=700
)

# Show the updated figure
fig.show()

# Save the interactive plot as an HTML file
html_file_path = "Downloads/interactive_3D_plot.html"
pio.write_html(fig, file=html_file_path, auto_open=False)

# Return the file path
html_file_path


In [ ]:
import numpy as np
import plotly.graph_objects as go

# Define the range for A, B, and C
A = np.linspace(0, 3, 50)
B = np.linspace(0, 3, 50)
C = np.linspace(0, 3, 50)

# Create a meshgrid for A, B, and C
A_3d, B_3d, C_3d = np.meshgrid(A, B, C, indexing='ij')

# Evaluate inequalities
inequality_1 = A_3d + B_3d >= np.log2(np.ceil(2**A_3d))
inequality_2 = A_3d + C_3d >= np.log2(np.ceil(2**A_3d))

# Union of both inequalities
valid_region = np.logical_or(inequality_1, inequality_2)

# Extract valid points (Red Region)
A_points = A_3d[valid_region]
B_points = B_3d[valid_region]
C_points = C_3d[valid_region]

# Find boundary (edges) where valid points meet invalid ones
edges = (
    (np.roll(valid_region, 1, axis=0) ^ valid_region) |  
    (np.roll(valid_region, 1, axis=1) ^ valid_region) |  
    (np.roll(valid_region, 1, axis=2) ^ valid_region)
)

# Extract edge points
A_edges = A_3d[edges]
B_edges = B_3d[edges]
C_edges = C_3d[edges]

# Filter black edges: Keep only points that directly touch the red region
touching_edges = valid_region[edges]

# Apply filtering
A_edges_filtered = A_edges[touching_edges]
B_edges_filtered = B_edges[touching_edges]
C_edges_filtered = C_edges[touching_edges]

# Marked Point (0.188649, 0.089036, 0.372337)
marked_A, marked_B, marked_C = 0.188649, 0.089036, 0.372337

# Create an interactive 3D scatter plot using Plotly
fig = go.Figure()

# Add valid region points (red)
fig.add_trace(go.Scatter3d(
    x=B_points, y=C_points, z=A_points,
    mode='markers',
    marker=dict(color='gray', size=3, opacity=0.3),
    name="Valid Region"
))

# Add only boundary-touching edges in black
fig.add_trace(go.Scatter3d(
    x=B_edges_filtered, y=C_edges_filtered, z=A_edges_filtered,
    mode='markers',
    marker=dict(color='black', size=3),
    name="Boundary Edges"
))

# Mark specific point in blue
fig.add_trace(go.Scatter3d(
    x=[marked_B], y=[marked_C], z=[marked_A],
    mode='markers',
    marker=dict(color='blue', size=8, symbol='x'),
    name="Marked Point (0.188649, 0.089036, 0.372337)"
))

# Set axis labels
fig.update_layout(
    scene=dict(
        xaxis_title="B",
        yaxis_title="C",
        zaxis_title="A"
    ),
    title="Interactive 3D Plot with Boundary Edges",
    width=800, height=700
)

# Show the interactive plot
fig.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
matplotlib.use('qt5agg')  # Use the Qt backend for interactivity
# Define the range for a, b, and c
a = np.linspace(0, 2, 50)  # Reduced resolution for efficiency
b = np.linspace(0, 2, 50)
c = np.linspace(0, 2, 50)

# Create a meshgrid
A, B, C = np.meshgrid(a, b, c, indexing='ij')

# Compute the two inequality conditions
inequality_1 = A + B >= np.log2(np.ceil(2**A))  # A + B >= log2(ceil(2^A))
inequality_2 = A + C >= np.log2(np.ceil(2**A))  # A + C >= log2(ceil(2^A))

# Combine the conditions using logical OR
valid_region = inequality_1 | inequality_2

# Extract the boundary points using differences in the boolean array
boundary = (
    (np.roll(valid_region, 1, axis=0) ^ valid_region) |  # Change along A
    (np.roll(valid_region, 1, axis=1) ^ valid_region) |  # Change along B
    (np.roll(valid_region, 1, axis=2) ^ valid_region)    # Change along C
)

# Extract points at the boundaries
A_edges = A[boundary]
B_edges = B[boundary]
C_edges = C[boundary]

# Create a 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot the boundary edges
ax.scatter(B_edges, C_edges,A_edges, c='r', alpha=0.5, label="Boundary Edges")

# Set labels and title
ax.set_xlabel(r'$B$', fontsize=12)
ax.set_ylabel(r'$C$', fontsize=12)
ax.set_zlabel(r'$A$', fontsize=12)
ax.set_title(r'Boundary of the Volume Captured by Inequalities', fontsize=14)

# Set axis limits
ax.set_xlim(0, 2)
ax.set_ylim(0, 2)
ax.set_zlim(0, 2)

# Add a legend
ax.legend()

# Show the plot
plt.show()
